In [661]:
print(f"InfluxDB2 client")

InfluxDB2 client


In [662]:
# InfluxDB Client docs : https://influxdb-python.readthedocs.io/en/latest/examples.html
# InfluxDB2 Client docs : https://influxdb-client.readthedocs.io/en/latest/
# InfluxDB2 Client Query docs : https://github.com/influxdata/influxdb-client-python/#queries

In [ ]:
#!pip install influxdb-client[async]

In [664]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import json
import random
import pytz
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [665]:
bucket = "SCRIPT"

In [666]:
def query_table(query_api):
    try:
        ## using Table structure
        tables = query_api.query('from(bucket:"{}") |> range(start: -5m)'.format(bucket))
        
        for table in tables:
            # print(table)
            for row in table.records:
                # print(json.dumps(row.values, indent=2))
                print(row.values)
    except Exception as e:
        print(e)

In [667]:
def query_dataframe(query_api):
    try:
        ## using Table structure
        data_frame  = query_api.query_data_frame('from(bucket:"{}") |> range(start: -1m)'.format(bucket))
        # print(data_frame.to_string())
        data_frame['_start'] = pd.to_datetime(data_frame['_start'], unit='s', utc=True).dt.tz_convert('America/Chicago').dt.tz_localize(None)
        data_frame['_stop'] = pd.to_datetime(data_frame['_stop'], unit='s', utc=True).dt.tz_convert('America/Chicago').dt.tz_localize(None)
        data_frame['_time'] = pd.to_datetime(data_frame['_time'], unit='s', utc=True).dt.tz_convert('America/Chicago').dt.tz_localize(None)
        display(data_frame)
        json_data = json.loads(data_frame.to_json(orient='records', date_format='iso'))
        print(json.dumps(json_data, indent=2))
    except Exception as e:
        print(e)

In [668]:
def query_csv(query_api):
    try:
        ## using csv library
        csv_result = query_api.query_csv('from(bucket:"SCRIPT") |> range(start: -5m)'.format(bucket))
        val_count = 0
        for row in csv_result:
            for cell in row:
                val_count += 1
    except Exception as e:
        print(e)

In [ ]:
try:
    # client = InfluxDBClient(url="https://localhost:8086", 
    #                         token="my-token", 
    #                         org="ES", 
    #                         verify_ssl=False)
    
    client = InfluxDBClient.from_config_file("influxDB2_config.ini")
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    sample_number = random.uniform(0,50)
    print(f"sample number : {sample_number}")
    p = Point("my_measurement").tag("location", "Prague").field("temperature", sample_number)
    
    write_api.write(bucket=bucket, record=p, write_precision='s')
except Exception as e:
    print(e)

sample number : 29.573676173795167


In [670]:
# query_table(query_api)
query_dataframe(query_api)

,result,table,_start,_stop,_time,_value,_field,_measurement,location
0,_result,0,2025-04-23 17:24:27.524647,2025-04-23 17:25:27.524647,2025-04-23 22:24:47.165699+00:00,39.642860,temperature,my_measurement,Prague
1,_result,0,2025-04-23 17:24:27.524647,2025-04-23 17:25:27.524647,2025-04-23 22:25:27.513850+00:00,29.573676,temperature,my_measurement,Prague


[
  {
    "result": "_result",
    "table": 0,
    "_start": "2025-04-23T17:24:27.524",
    "_stop": "2025-04-23T17:25:27.524",
    "_time": "2025-04-23T22:24:47.165Z",
    "_value": 39.6428601363,
    "_field": "temperature",
    "_measurement": "my_measurement",
    "location": "Prague"
  },
  {
    "result": "_result",
    "table": 0,
    "_start": "2025-04-23T17:24:27.524",
    "_stop": "2025-04-23T17:25:27.524",
    "_time": "2025-04-23T22:25:27.513Z",
    "_value": 29.5736761738,
    "_field": "temperature",
    "_measurement": "my_measurement",
    "location": "Prague"
  }
]
